# Agentic RAG

In [1]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from datetime import datetime
from langchain.tools import tool

today = datetime.today().strftime("%B ") + str(datetime.today().day) + ("th" if 11<=datetime.today().day<=13 else {1:"st",2:"nd",3:"rd"}.get(datetime.today().day%10,"th")) + datetime.today().strftime(", %Y")
print(today)

@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"

tools = [search]

llm = ChatOpenAI(model="gpt-4.1")
agent_executor = create_agent(
    llm, tools, system_prompt=f'Today is {today}')  # true as of today :)


/Users/sinanozdemir/Teaching/Pearson/oreilly-ai-pipelines/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


February 4th, 2026


In [2]:
from langchain.messages import HumanMessage

print(agent_executor.invoke({"messages": HumanMessage('how do I replace my medicare card')})['messages'][-1].content)

To replace your Medicare card, you generally have several options:

1. Online: Visit the official Medicare website (medicare.gov or, if you’re in Australia, servicesaustralia.gov.au) and log in to your account. There should be an option to request a replacement card.

2. By Phone: Call Medicare's customer service line. For U.S. residents, the number is 1-800-MEDICARE (1-800-633-4227). For Australians, call the Medicare general enquiries line.

3. In Person: Visit a local Social Security office (U.S.) or a Medicare Service Centre (Australia) to request a replacement in person.

You will typically need your Medicare number or Social Security number for verification.

Would you like step-by-step instructions for your specific country, or help with the online process?


In [6]:
import os
pinecone_key = os.environ.get('PINECONE_API_KEY')
INDEX_NAME = 'rag-basics'
ENGINE = 'text-embedding-3-small'
NAMESPACE = 'default'

In [7]:
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI

pc = Pinecone(
    api_key=pinecone_key
)
# Store the index as a variable
index = pc.Index(name=INDEX_NAME)
index

client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

# helper functions to get lists of embeddings from the OpenAI API
def get_embedding(text, engine=ENGINE):
    response = client.embeddings.create(
        input=[text],
        model=engine
    )
    return response.data[0].embedding

len(get_embedding('hi'))

@tool
def query_from_pinecone(query, top_k=1, include_metadata=True):
    '''Use this tool to look things up about Medicaid, SSA, etc'''
    query_embedding = get_embedding(query, engine=ENGINE)

    return index.query(
      vector=query_embedding,
      top_k=top_k,
      namespace=NAMESPACE,
      include_metadata=include_metadata   # gets the metadata (dates, text, etc)
    ).get('matches')



In [8]:
agent_executor = create_agent(
    llm, [query_from_pinecone], system_prompt=f'Today is {today}')  # true as of today :)


print(agent_executor.invoke({"messages": HumanMessage('how do I replace my medicare card')})['messages'][-1].content)

If your Medicare card was lost, stolen, or destroyed, you can request a replacement in one of these ways:

1. Online: Go to Medicare.gov and sign into your online Medicare account. You can view, download, or print an official copy of your Medicare card right away.

2. By Phone: Call 1-800-MEDICARE (1-800-633-4227) to order a replacement card that will be sent to you by mail.

Let me know if you’d like more specific step-by-step instructions for either method!


In [9]:
response = agent_executor.invoke({"messages": HumanMessage('how do I replace my medicare card')})['messages']

In [17]:
response[-4]

HumanMessage(content='how do I replace my medicare card', additional_kwargs={}, response_metadata={}, id='3e63c22f-5e8b-407a-9328-3235c4af7a03')

In [19]:
type(response[-3]), response[-3].content

(langchain_core.messages.ai.AIMessage, '')

In [15]:
response[-3].tool_calls

[{'name': 'query_from_pinecone',
  'args': {'query': 'how to replace Medicare card'},
  'id': 'call_tz4UNhnz3V2w4L5VOUxpccLZ',
  'type': 'tool_call'}]

In [11]:
response[-2]

ToolMessage(content="[{'id': '0462ed04ee959c4413426cd1f8307f9c',\n 'metadata': {'date_uploaded': '2025-10-21T16:26:39.478842+00:00',\n              'text': 'KA01735 How do I get a replacement Medicare card? If '\n                      'your Medicare card was lost, stolen, or destroyed, you '\n                      'can request a replacement online at Medicare.gov. You '\n                      'can an official copy of your card from your online '\n                      'Medicare account or call 1800MEDICARE to order a '\n                      'replacement card to be sent in the mail.',\n              'url': 'http://faq.ssa.gov/en-us/topic/article/ka-01735'},\n 'score': 0.713479042,\n 'values': []}]", name='query_from_pinecone', id='8a5e6720-17f5-4589-92c9-14ca8aecdc89', tool_call_id='call_tz4UNhnz3V2w4L5VOUxpccLZ')

In [13]:
response[-1].content

'If your Medicare card was lost, stolen, or destroyed, you can request a replacement in one of two ways:\n\n1. Online: Go to Medicare.gov and log in to your online Medicare account to get an official copy of your card.\n2. By phone: Call 1-800-MEDICARE (1-800-633-4227) to order a replacement card to be sent to you by mail.\n\nLet me know if you need step-by-step instructions for either option!'